## Лекция 2  BM5    

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
# инициализируем
vectorizer = TfidfVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
 
# получится следующая структура:
#        |  слово1  |  слово2  |  слово3  |  слово4
# текст1 |   0.6    |    0.5   |   0.6    |    0
# текст2 |   0      |    0.6   |   0.8    |    0
# текст3 |   0.9    |    0.4   |   0      |    0
# текст4 |   0      |    0     |   0      |    1
 
# чтобы получить сгенерированный словарь, из приведенной структуры TfidfVectorizer
# порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
vectorizer.vocabulary_.get('слово3') # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
new_doc = vectorizer.transform(['слово1 слово4 слово4']).toarray()  # результат [[0.36673901, 0, 0, 0.93032387]]
 

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [ ]:
### реализуйте эту функцию ранжирования 
from math import log
import statistics
import re

k = 2.0
b = 0.75

collection = ['Они зашли в дом. В доме было пусто.',
              'Передо мной сидел рыжий котик. У него был длинный хвост.',
              'Я выглянул в окно. Мимо ехал синий автобус.',
              'Мы вышли на улицу. Что-то было явно не так.',
              'На столе стояла ваза. В ней были цветы.',
              'Надо найти заправку. Топлива почти нет.',
              'Автобус проехал под железнодорожным мостом. Наверху стоял поезд.',
              'В школе начался урок. Все сели за парты и достали свои тетрадки.']

collection_length = []
n_big = len(collection)

for c in collection:
  collection_length.append(len(c))

avgdl = statistics.mean(collection_length)

# d - один документ
# q - набор слов

def bm25(d,q) -> float:
  l_d = len(d)
  def frequency(single_q):
    tf_q_d = 0
    for word in d.split():
      word = re.sub('!@#$%^&*\(\)\[\]\{\}.,><?/-/"',' ',word)
      word = word.lower()
      #print(word)
      #print(single_q)
      if single_q == word:
        tf_q_d += 1
    return tf_q_d
  
  def get_idf(single_q):
    
    def number_of_docs(another_q):
      doc_num = 0
      for col in collection:
        if another_q in col:
          doc_num += 1
      return doc_num
  
    end_result = log((n_big - number_of_docs(single_q) + 0.5)/
                     (number_of_docs(single_q) + 0.5))
    return end_result
  
  final_meanings = []

  for query in q:
    #print(query)
    upper_half = frequency(query) * (k + 1)
    lower_half = frequency(query) + k * (1 - b + b * (l_d)/avgdl)
    final_meanings.append(get_idf(query) * upper_half/lower_half)

  return sum(final_meanings)

  
  #query_idfs = []
  #for query in q:
    #query_idfs.append(get_idf(query))
  
  #for qq in q:
    
  #def tf(element):
  #  d
  #n_qi = 0
  #for d in documents:
  #def idf(q_i):
  #  idf_qi = log((N - n_qi + 0.5)/n_qi + 0.5)
  #  return 

In [ ]:
lis = ['в','было','автобус']

for coll in collection:
  print(bm25(coll,lis))

-3.6225241954868634
0.0
-2.9852014051541427
1.0067699682132274
-3.122649527405844
0.0
1.3774630624854982
-2.4248507503319128


### __Задача 1__:    
Реализуйте поиск с метрикой *TF-IDF* через умножение матрицы на вектор.
Что должно быть в реализации:
- проиндексированная база, где каждый документ представлен в виде вектора TF-IDF
- функция перевода входяшего запроса в вектор по метрике TF-IDF
- ранжирование докуменов по близости к запросу по убыванию

В качестве корпуса возьмите корпус вопросов в РПН по Covid2019. Он состоит из:
> файл **answers_base.xlsx** - база ответов, у каждого ответа есть его номер, тематика и примеры вопросов, которые могут быть заданы к этому ответу. Сейчас проиндексировать надо именно примеры вопросов в качестве документов базы. Понимаете почему?

> файл **queries_base.xlsx** - вопросы юзеров, к каждому из которых проставлен номер верного ответа из базы. Разделите эти вопросы в пропорции 70/30 на обучающую (проиндексированную как база) и тестовую (как запросы) выборки. 


In [ ]:
#import openpyxl
#from pathlib import Path
import pandas as pd
import unicodedata
#from sklearn.feature_extraction.text import TfidfVectorizer

#answers = openpyxl.load_workbook('answers_base.xlsx')
#queries = openpyxl.load_workbook('queries_base.xlsx')

#answers_sheet = answers.active
#queries_sheet = queries.active

answers = pd.read_excel('answers_base.xlsx')
queries = pd.read_excel('queries_base.xlsx')
#base_1 = 

In [ ]:
queries

,Текст вопроса,Номер связки\n,Тематика,Unnamed: 3,Unnamed: 4
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,NaN,NaN
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,NaN,NaN
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",NaN,NaN
...,...,...,...,...,...
2294,Добрый день.\nГражданин Украины проживает в РФ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2295,Добрый вечер. Нахожусь в командировке за грани...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",NaN,NaN
2296,Здравствуйте! \nНам пришёл положительный резул...,1.0,"КАРАНТИН, ИЗОЛЯЦИЯ, САМОИЗОЛЯЦИЯ\n",NaN,NaN
2297,"Добрый день, хотелось бы получить консультацию...",45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ\n",NaN,NaN


In [ ]:
#queries['Номер связки\n']
#question_corpus = []
#for q in queries['Текст вопроса']:
#  q = re.sub('\n', ' ', str(q))
#  q = re.sub('\xad', ' ', q)
#  q = unicodedata.normalize("NFKD", q)
#  #q = re.sub('\xa', ' ', q)
#  question_corpus.append(q)
#print(question_corpus)
#print(len(question_corpus))

['с уважением Вероника Игоревна  Ильич  После 15 августа 2020 года к нам, в Москву планируют приехать гости из Молдовы. Какой на сегодняшний день режим по Ковид распространяется на иностранцев, прибывающих в Россию?  С уважением, Иван Звягинцев ', 'Здравствуйте! Проинформируйте, пожалуйста, нужен ли тест на COVID-19 при заселении в санаторий в августе 2020 года? ', '-- Добрый день!    Меня, Сидельникова Андрея Олеговича 30.07.1989г.р., посадили на карантин как контактного. После выяснения у заболевшего коронавирус не подтвердился. На работе мне нужно подтвердить документально мое отсутствие с 3.08 по 07.08.2020. Обратился в инфекционисту, на что та сказала, что за свой счет, либо досиживайте по Предписанию все положенные 14 дней. Находиться на больничном до 14.08.2020 не имею финансовой возможности, да и на предприятии к этому не очень хорошо относяться. Вопрос! Почему я должен за свой счет закрывать дни карантина. Прошу как можно быстрее разобраться в сложившейся ситуации

In [ ]:
a_corpus = []
q_corpus = []
for q in answers['Текст вопросов']:
  q = re.sub('\n', ' ', str(q))
  q = re.sub('\xad', ' ', q)
  q = unicodedata.normalize("NFKD", q)
  #q = re.sub('\xa', ' ', q)
  q_corpus.append(q)

#queries['']

for a in queries['Текст вопроса']:
  a = re.sub('\n', ' ', str(a))
  a = re.sub('\xad', ' ', a)
  a = unicodedata.normalize("NFKD", a)
  #q = re.sub('\xa', ' ', q)
  a_corpus.append(a)

In [ ]:
#X = vectorizer.fit_transform(answers['Текст вопросов'])
#queries['Текст вопроса']

0       с уважением Вероника Игоревна  Ильич\n\nПосле ...
1       Здравствуйте! Проинформируйте, пожалуйста, нуж...
2       --\nДобрый день!\n   Меня, Сидельникова Андрея...
3       Добрый день.\nВ Кемеровской области согласно п...
4       Здравствуйте, в моем городе Кострома введено о...
                              ...                        
2294    Добрый день.\nГражданин Украины проживает в РФ...
2295    Добрый вечер. Нахожусь в командировке за грани...
2296    Здравствуйте! \nНам пришёл положительный резул...
2297    Добрый день, хотелось бы получить консультацию...
2298    Добрый день ! \nМоя мама Уланина Валентина Мих...
Name: Текст вопроса, Length: 2299, dtype: object

In [ ]:
#X = vectorizer.fit_transform(q_corpus)
X = vectorizer.fit_transform(answers['Текст вопросов'].values.astype('U'))
Y = vectorizer.transform(queries['Текст вопроса'].values.astype('U')).toarray()
#print(Y)
Y_t = Y.transpose()
#queries_t = X.toarray().transpose()
#for d in question_corpus:
print(X.dot(Y_t))

[[0.05132964 0.03686202 0.08700867 ... 0.03924478 0.08545746 0.01458394]
 [0.00826135 0.02554302 0.04825987 ... 0.         0.04846608 0.1205976 ]
 [0.         0.00846719 0.01285142 ... 0.         0.00300929 0.        ]
 ...
 [0.02310699 0.01857331 0.15761451 ... 0.00454906 0.05848957 0.03722858]
 [0.02007746 0.00861717 0.0528506  ... 0.00563377 0.02723117 0.01461121]
 [0.01955076 0.         0.06385532 ... 0.         0.02984706 0.        ]]


In [ ]:
X_q = vectorizer.fit_transform(answers['Текст вопросов'])
for q in X_q.toarray():
  m = []
  for i, l in enumerate(X_q.toarray()):
    m.append((i, l * q.T))

In [ ]:
#m

[(0, array([0., 0., 0., ..., 0., 0., 0.])),
 (1, array([0., 0., 0., ..., 0., 0., 0.])),
 (2, array([0., 0., 0., ..., 0., 0., 0.])),
 (3, array([0., 0., 0., ..., 0., 0., 0.])),
 (4, array([0., 0., 0., ..., 0., 0., 0.])),
 (5, array([0., 0., 0., ..., 0., 0., 0.])),
 (6, array([0., 0., 0., ..., 0., 0., 0.])),
 (7, array([0., 0., 0., ..., 0., 0., 0.])),
 (8, array([0., 0., 0., ..., 0., 0., 0.])),
 (9, array([0., 0., 0., ..., 0., 0., 0.])),
 (10, array([0., 0., 0., ..., 0., 0., 0.])),
 (11, array([0., 0., 0., ..., 0., 0., 0.])),
 (12, array([0., 0., 0., ..., 0., 0., 0.])),
 (13, array([0., 0., 0., ..., 0., 0., 0.])),
 (14, array([0., 0., 0., ..., 0., 0., 0.])),
 (15, array([0., 0., 0., ..., 0., 0., 0.])),
 (16, array([0., 0., 0., ..., 0., 0., 0.])),
 (17, array([0., 0., 0., ..., 0., 0., 0.])),
 (18, array([0., 0., 0., ..., 0., 0., 0.])),
 (19, array([0., 0., 0., ..., 0., 0., 0.])),
 (20, array([0., 0., 0., ..., 0., 0., 0.])),
 (21, array([0., 0., 0., ..., 0., 0., 0.])),
 (22, array([0., 0.,

In [ ]:
#for mm in m:
#  print(X.dot(Y_t))

[[0.0420838  0.02847641 0.09157133 ... 0.04825647 0.08879984 0.01458394]
 [0.00846442 0.01973233 0.05079058 ... 0.         0.05036166 0.1205976 ]
 [0.         0.00654102 0.01352534 ... 0.         0.00312699 0.        ]
 ...
 [0.01715136 0.0472393  0.17346539 ... 0.02469793 0.06077719 0.03722858]
 [0.02057099 0.00665688 0.05562204 ... 0.00692744 0.02829622 0.01461121]
 [0.02003134 0.         0.06720384 ... 0.         0.03101443 0.        ]]
[[0.0420838  0.02847641 0.09157133 ... 0.04825647 0.08879984 0.01458394]
 [0.00846442 0.01973233 0.05079058 ... 0.         0.05036166 0.1205976 ]
 [0.         0.00654102 0.01352534 ... 0.         0.00312699 0.        ]
 ...
 [0.01715136 0.0472393  0.17346539 ... 0.02469793 0.06077719 0.03722858]
 [0.02057099 0.00665688 0.05562204 ... 0.00692744 0.02829622 0.01461121]
 [0.02003134 0.         0.06720384 ... 0.         0.03101443 0.        ]]
[[0.0420838  0.02847641 0.09157133 ... 0.04825647 0.08879984 0.01458394]
 [0.00846442 0.01973233 0.05079058 ... 

In [ ]:
#for q, a in ['Номер связки\n	'], answers['Номер связки']:
#  print(q)
#print(X)
#queries['Текст вопроса']

  (0, 4535)	0.28836628857414637
  (0, 4634)	0.243476094772584
  (0, 11435)	0.13990554580723835
  (0, 10075)	0.1847957396088007
  (0, 4845)	0.243476094772584
  (0, 11092)	0.23447075723119062
  (0, 5368)	0.10072912140564773
  (0, 8999)	0.06897930501901349
  (0, 11231)	0.162006385427964
  (0, 3446)	0.06017535050447862
  (0, 11829)	0.2008497874437889
  (0, 6657)	0.08800908520665544
  (0, 5105)	0.1413564707660806
  (0, 6545)	0.2555328878696153
  (0, 4670)	0.0618422999354713
  (0, 3112)	0.2212940932807161
  (0, 10174)	0.185805185256858
  (0, 8940)	0.243476094772584
  (0, 6580)	0.15401049353462795
  (0, 6763)	0.12938438917609088
  (0, 3035)	0.14643866517978263
  (0, 235)	0.11015900871536664
  (0, 1125)	0.11193831504425893
  (0, 101)	0.1653163940625227
  (0, 9642)	0.10991188393732435
  :	:
  (2298, 14145)	0.1472455182913836
  (2298, 9938)	0.15536535103132762
  (2298, 8994)	0.1490339105362453
  (2298, 6078)	0.15306530177259386
  (2298, 6477)	0.1472455182913836
  (2298, 1938)	0.1720925533313693


### __Задача 2__:    
Аналогичная задаче1 с другой метрикой 

Реализуйте поиск с метрикой *BM25* через умножение матрицы на вектор. Что должно быть в реализации:

- проиндексированная база, где каждый документ представлен в виде вектора BM25
- функция перевода входяшего запроса в вектор по метрике BM25
- ранжирование докуменов по близости к запросу по убыванию

In [ ]:
for coll in collection:
  print(bm25(X,Y))

AttributeError: ignored